In [ ]:
##Don't run unless on a desktop, create Virtual env
##python -m venv env
##.\env\Scripts\activate

In [ ]:
!pip install langchain==0.0.184
!pip install PyPDF2==3.0.1
!pip install python-dotenv==1.0.0
!pip install streamlit==1.18.1
!pip install openai==0.27.6
!pip install faiss-cpu==1.7.4
!pip install altair==4
!pip install tiktoken==0.4.0
!pip install python-dotenv
!pip install langchain-community
!npm install localtunnel

  Using cached langchain-0.0.184-py3-none-any.whl.metadata (12 kB)
Using cached langchain-0.0.184-py3-none-any.whl (939 kB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.14
    Uninstalling langchain-0.2.14:
      Successfully uninstalled langchain-0.2.14
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.2.12 requires langchain<0.3.0,>=0.2.13, but you have langchain 0.0.184 which is incompatible.
  Using cached langchain-0.2.14-py3-none-any.whl.metadata (7.1 kB)
Using cached langchain-0.2.14-py3-none-any.whl (997 kB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.184
    Uninstalling langchain-0.0.184:
      Successfully uninstalled langchain-0.0.184

up to date, audited 23 packages in 660ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severi

In [ ]:
%%writefile app.py


import streamlit as st
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import sys
sys.path.append('/content/drive/MyDrive/ML/Projects/3_Chatbot- LangChain & GPT-4')
from htmlTemplates import css, bot_template, user_template
from langchain.llms import HuggingFaceHub
import os

os.environ['OPENAI_API_KEY'] = ""

def get_pdf_text(pdf_docs):
    text = ""
    pdf_reader = PdfReader(pdf_docs)
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

def get_vectorstore(text_chunks):
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore

def get_conversation_chain(vectorstore):
    llm = ChatOpenAI()

    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain

def handle_userinput(user_question):
    response = st.session_state.conversation({'question': user_question})
    st.session_state.chat_history = response['chat_history']

    for i, message in enumerate(st.session_state.chat_history):
        if i % 2 == 0:
            st.write(user_template.replace(
                "{{MSG}}", message.content), unsafe_allow_html=True)
        else:
            st.write(bot_template.replace(
                "{{MSG}}", message.content), unsafe_allow_html=True)

def main():
    load_dotenv()
    st.set_page_config(page_title="Chat with your PDF",
                       page_icon=":book:")
    st.write(css, unsafe_allow_html=True)

    if "conversation" not in st.session_state:
        st.session_state.conversation = None
    if "chat_history" not in st.session_state:
        st.session_state.chat_history = None

    st.header("Chat with your PDF :book:")
    user_question = st.text_input("Ask a question about your documents:")
    if user_question:
        handle_userinput(user_question)

    with st.sidebar:
        pdf_docs = open('/content/drive/MyDrive/ML/Projects/3_Chatbot- LangChain & GPT-4/docs/tsla-20231231-gen.pdf', 'rb')
        raw_text = get_pdf_text(pdf_docs)
        # get the text chunks
        text_chunks = get_text_chunks(raw_text)
        # create vector store
        vectorstore = get_vectorstore(text_chunks)
        # create conversation chain
        st.session_state.conversation = get_conversation_chain(
                    vectorstore)


if __name__ == '__main__':
    main()

Overwriting app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.45.20.75
your url is: https://smart-bikes-follow.loca.lt
